In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
from delta import *

# warehouse_location points to the default location for managed databases and tables
warehouse_location = 'hdfs://hdfs-nn:9000/warehouse'

builder = SparkSession \
    .builder \
    .appName("Python Spark SQL Hive integration example") \
    .config("spark.sql.warehouse.dir", warehouse_location) \
    .config("hive.metastore.uris", "thrift://hive-metastore:9083") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:1.0.0") \
    .enableHiveSupport() \

spark = spark = configure_spark_with_delta_pip(builder).getOrCreate()

In [2]:
spark.sql(
    """
   SHOW tables from PL7G1_GOLD
   """
).toPandas()

database                     tableName  isTemporary
0  pl7g1_gold             air_quality_gases        False
1  pl7g1_gold      air_quality_gases_presto        False
2  pl7g1_gold             reciclagem_presto        False
3  pl7g1_gold         tabela_individual_air        False
4  pl7g1_gold  tabela_individual_air_presto        False
5  pl7g1_gold         tabela_querie4_presto        False
6  pl7g1_gold                tabela_querie5        False
7  pl7g1_gold         tabela_querie5_presto        False

In [3]:
air_quality = spark.table("PL7G1_GOLD.Air_Quality_Gases")
reciclagem = spark.table("PL7G1_GOLD.Reciclagem")

AnalysisException: Table or view not found: PL7G1_GOLD.Reciclagem;
'UnresolvedRelation [PL7G1_GOLD, Reciclagem], [], false


In [ ]:
instalacoes_Reciclagem = reciclagem.join(air_quality, (reciclagem.Year == air_quality.Year) & (reciclagem.Borough == air_quality.Borough),"inner").drop(reciclagem.Borough)

In [ ]:
instalacoes_Reciclagem =instalacoes_Reciclagem.drop(air_quality.Year)

In [ ]:
instalacoes_Reciclagem.printSchema()

In [ ]:
spark.sql(
    """
    CREATE EXTERNAL TABLE PL7G1_GOLD.Air_Quality_Reciclagem (
        Year INT,
        Borough VARCHAR(50),
        PM2_5_SUM Double,
        PM2_5_AVG Double,
        PM2_5_MIN Double,
        PM2_5_MAX Double,
        NO2_SUM Double,
        NO2_AVG Double,
        NO2_MIN Double,
        NO2_MAX Double,
        O3_SUM Double,
        O3_AVG Double,
        O3_MIN Double,
        O3_MAX Double,
        SO2_SUM Double,
        SO2_AVG Double,
        SO2_MIN Double,
        SO2_MAX Double,
        SUM_Capture_Rate_Paper Double,
        AVG_Capture_Rate_Paper Double,
        MAX_Capture_Rate_Paper Double,
        MIN_Capture_Rate_Paper Double,
        SUM_Capture_Rate_MGP Double,
        AVG_Capture_Rate_MGP Double,
        MAX_Capture_Rate_MGP Double,
        MIN_Capture_Rate_MGP Double,
        SUM_Capture_Rate_Total Double,
        AVG_Capture_Rate_Total Double,
        MAX_Capture_Rate_Total Double,
        MIN_Capture_Rate_Total Double
    )
    USING DELTA
    LOCATION 'hdfs://hdfs-nn:9000/Poluicao_ar/Gold/PL7G1_GOLD.db/Air_Quality_Reciclagem'
    """
)

In [ ]:
instalacoes_Reciclagem = instalacoes_Reciclagem \
    .write \
    .format("delta") \
    .mode("overwrite") \
    .option("mergeSchema", "true")\
    .save("hdfs://hdfs-nn:9000/Poluicao_ar/Gold/PL7G1_GOLD.db/Air_Quality_Reciclagem")

In [ ]:
spark.table("PL7G1_GOLD.Air_Quality_Reciclagem").toPandas()

In [ ]:
spark.sql("""
GENERATE symlink_format_manifest FOR TABLE delta.`hdfs://hdfs-nn:9000/Poluicao_ar/Gold/PL7G1_GOLD.db/Air_Quality_Reciclagem/`
""").show()

In [ ]:
spark.sql("""
    CREATE EXTERNAL TABLE IF NOT EXISTS PL7G1_GOLD.Air_Quality_Reciclagem_presto (
        Year INT,
        Borough VARCHAR(50),
        PM2_5_SUM Double,
        PM2_5_AVG Double,
        PM2_5_MIN Double,
        PM2_5_MAX Double,
        NO2_SUM Double,
        NO2_AVG Double,
        NO2_MIN Double,
        NO2_MAX Double,
        O3_SUM Double,
        O3_AVG Double,
        O3_MIN Double,
        O3_MAX Double,
        SO2_SUM Double,
        SO2_AVG Double,
        SO2_MIN Double,
        SO2_MAX Double,
        SUM_Capture_Rate_Paper Double,
        AVG_Capture_Rate_Paper Double,
        MAX_Capture_Rate_Paper Double,
        MIN_Capture_Rate_Paper Double,
        SUM_Capture_Rate_MGP Double,
        AVG_Capture_Rate_MGP Double,
        MAX_Capture_Rate_MGP Double,
        MIN_Capture_Rate_MGP Double,
        SUM_Capture_Rate_Total Double,
        AVG_Capture_Rate_Total Double,
        MAX_Capture_Rate_Total Double,
        MIN_Capture_Rate_Total Double
    )
    ROW FORMAT SERDE 'org.apache.hadoop.hive.ql.io.parquet.serde.ParquetHiveSerDe'
    STORED AS INPUTFORMAT 'org.apache.hadoop.hive.ql.io.SymlinkTextInputFormat'
    OUTPUTFORMAT 'org.apache.hadoop.hive.ql.io.HiveIgnoreKeyTextOutputFormat'
    LOCATION 'hdfs://hdfs-nn:9000/Poluicao_ar/Gold/PL7G1_GOLD.db/Air_Quality_Reciclagem/_symlink_format_manifest/'
    """).show()